# The purpose of this workbook is to show the tests cases executed


[Contract Reference from David](https://github.com/fetchai/agents-aea/blob/10bc452827487403d65c6c4bc6b3ad985c269997/docs/contract.md)

# Understanding of The flow

The ledger API connection skill handles the [ContractApiMessage]() object

The [Contract](https://github.com/fetchai/agents-aea/blob/10bc452827487403d65c6c4bc6b3ad985c269997/aea/contracts/base.py)
class handles the ContractAPIMessage. 

The contract class we implement within the AEA contract.py  file within the contract class of the AEA inheirts from this object, so we therefore have all of its available methods

Based on the referenced documentation, we would need to impl

Contract API Message - The contract API message we must base our message on is of the RAW_TRANSACTION_TYPE


```
contract_api_msg = ContractApiMessage(
    performative=ContractApiMessage.Performative.GET_RAW_TRANSACTION,
    dialogue_reference=contract_api_dialogues.new_self_initiated_dialogue_reference(),
    ledger_id=strategy.ledger_id,
    contract_id="fetchai/erc1155:0.9.0",
    contract_address=strategy.contract_address,
    callable="get_create_batch_transaction",
    kwargs=ContractApiMessage.Kwargs(
        {
            "deployer_address": self.context.agent_address,
            "token_ids": strategy.token_ids,
        }
    ),
)
```
Our object must therefore look like this;


```
contract_api_msg = ContractApiMessage(
    performative=ContractApiMessage.Performative.GET_RAW_TRANSACTION,
    dialogue_reference=contract_api_dialogues.new_self_initiated_dialogue_reference(),
    ledger_id=strategy.ledger_id,
    contract_id="tomrae/ethpool:0.1.0",
    contract_address=self.context.strategy.deployment_status["ethpool"][1], 
    callable="provide_liquidity",
    kwargs=ContractApiMessage.Kwargs(
        {
            "deployer_address": self.context.agent_address,
            "amount": 10000,
        }
    ),
)
```
note I have cross referenced this address, 
so I know that the agent is correctly deploying the 
transactions as this address matches the address displayed on ganache CLI, under contract address

In [4]:
# Now, this will get passed to the provide liquidity function of my own contract.py

In [ ]:
    @classmethod
    def provide_liquidity(
            cls,
            ledger_api: LedgerApi,
            contract_address: Address,
            deployer_address: Address,
            amount: int,
            gas: int = 60000000,
    ) -> Dict[str, Any]:
        """
        Get the transaction to create a batch of tokens.

        :param ledger_api: the ledger API
        :param deployer_address: the address of the deployer
        :param amount: the arguments
        :param gas: the gas to be used
        :return: the transaction object
        """
        logger.info(f"******************************** just about to creat contract {deployer_address} type {type(deployer_address)}")
        nonce = ledger_api.api.eth.getTransactionCount(deployer_address)
        instance = cls.get_instance(ledger_api, contract_address)
        logger.info(f"************instance created:")
        tx = instance.functions.provide(amount).buildTransaction({
            "gas": gas,
            "gasPrice":ledger_api.api.toWei("50", "gwei"),
            "nonce": nonce,
        })
        logger.info(f"************tx created:")
        tx = cls._try_estimate_gas(ledger_api, tx)
        logger.info(f"************ gas estimated:")

        return tx  #return #

In [5]:
# Using the existing method from the erc1155 contract as inspiration, 

In [ ]:
# This is based on the following method
   @classmethod
    def get_create_batch_transaction(
        cls,
        ledger_api: LedgerApi,
        contract_address: Address,
        deployer_address: Address,
        token_ids: List[int],
        data: Optional[bytes] = b"",
        gas: int = 300000,
    ) -> Dict[str, Any]:
        """
        Get the transaction to create a batch of tokens.
        :param ledger_api: the ledger API
        :param contract_address: the address of the contract
        :param deployer_address: the address of the deployer
        :param token_ids: the list of token ids for creation
        :param data: the data to include in the transaction
        :param gas: the gas to be used
        :return: the transaction object
        """
        # create the transaction dict
        nonce = ledger_api.api.eth.getTransactionCount(deployer_address)
        instance = cls.get_instance(ledger_api, contract_address)
        tx = instance.functions.createBatch(
            deployer_address, token_ids
        ).buildTransaction(
            {
                "gas": gas,
                "gasPrice": ledger_api.api.toWei("50", "gwei"),
                "nonce": nonce,
            }
        )
        tx = cls._try_estimate_gas(ledger_api, tx)
        return tx

Which fails with

```
INFO:aea.packages.tomrae.skills.option_monitoring:[autonomous_hegician] requesting provide liquidity transaction...
info: ******* Just about to get nonce for 0xD9EEA0deAC1cf2Ae88Cb2E9fBfa0De8A61F9CF7C type <class 'str'>
INFO:aea.packages.fetchai.contracts.ethpool:******* Just about to get nonce for 0xD9EEA0deAC1cf2Ae88Cb2E9fBfa0De8A61F9CF7C type <class 'str'>
info: [autonomous_hegician] Monitoring 4 orders for execution.
INFO:aea.packages.tomrae.skills.option_monitoring:[autonomous_hegician] Monitoring 4 orders for execution.
info: ******* Just about to get instance 
INFO:aea.packages.fetchai.contracts.ethpool:******* Just about to get instance 
error: [autonomous_hegician] An error occurred while processing the contract api request: '('Web3.py only accepts checksum addresses. The software that gave you this non-checksum address should be considered unsafe, please file it as a bug on their platform. Try using an ENS name instead. Or, if you must accept lower safety, use Web3.toChecksumAddress(lower_case_address).', '0x1b027efaabf20c82886f988da9e3595bed37687e')'.
ERROR:aea.packages.fetchai.connections.ledger:[autonomous_hegician] An error occurred while processing the contract api request: '('Web3.py only accepts checksum addresses. The software that gave you this non-checksum address should be considered unsafe, please file it as a bug on their platform. Try using an ENS name instead. Or, if you must accept lower safety, use Web3.toChecksumAddress(lower_case_address).', '0x1b027efaabf20c82886f988da9e3595bed37687e')'.
info: [autonomous_hegician] Current Rate : 384.28
INFO:aea.packages.tomrae.skills.option_monitoring:[autonomous_hegician] Current Rate : 384.28
info: [autonomous_hegician] received ledger_api error message=Message(dialogue_reference=('10dde1696fb8616105a1c6b5d4390159b14ea8adeb623e7e7015762e57ec60b9', '32c6aeabe4c2298c1b98c726f81f562c5a2e9fea1dfd30f5c796a182fbee84a9') message_id=2 target=1 performative=error code=500 message=('Web3.py only accepts checksum addresses. The software that gave you this non-checksum address should be considered unsafe, please file it as a bug on their platform. Try using an ENS name instead. Or, if you must accept lower safety, use Web3.toChecksumAddress(lower_case_address).', '0x1b027efaabf20c82886f988da9e3595bed37687e') data=b'') in dialogue=Dialogue Label: 10dde1696fb8616105a1c6b5d4390159b14ea8adeb623e7e7015762e57ec60b9_32c6aeabe4c2298c1b98c726f81f562c5a2e9fea1dfd30f5c796a182fbee84a9_fetchai/ledger:0.3.0_0xD9EEA0deAC1cf2Ae88Cb2E9fBfa0De8A61F9CF7C
info: get_raw_transaction( )
info: error( ).
INFO:aea.packages.tomrae.skills.option_monitoring:[autonomous_hegician] received ledger_api error message=Message(dialogue_reference=('10dde1696fb8616105a1c6b5d4390159b14ea8adeb623e7e7015762e57ec60b9', '32c6aeabe4c2298c1b98c726f81f562c5a2e9fea1dfd30f5c796a182fbee84a9') message_id=2 target=1 performative=error code=500 message=('Web3.py only accepts checksum addresses. The software that gave you this non-checksum address should be considered unsafe, please file it as a bug on their platform. Try using an ENS name instead. Or, if you must accept lower safety, use Web3.toChecksumAddress(lower_case_address).', '0x1b027efaabf20c82886f988da9e3595bed37687e') data=b'') in dialogue=Dialogue Label: 10dde1696fb8616105a1c6b5d4390159b14ea8adeb623e7e7015762e57ec60b9_32c6aeabe4c2298c1b98c726f81f562c5a2e9fea1dfd30f5c796a182fbee84a9_fetchai/ledger:0.3.0_0xD9EEA0deAC1cf2Ae88Cb2E9fBfa0De8A61F9CF7C
get_raw_transaction( )
error( ).
```

In [7]:
# /\ this suggests that the deployer address is ok as the nonce is calculated, yet for some reason the Contract address is NOT!


In [15]:
cp defi_cefi_bridge/tontine/hedge-contracts-v1/con

/home/tom/Desktop/Medium


In [11]:
# in order to that A), the contract address is correct and B) that the contract is getting deployed with no issues,

# I restart ganache, and remove the contract address for the eth pools from the skill config. This will cause the agent to redeploy the contracts when it starts up.

# note, I am reading in the json directly to deploy, (as I know it works)

# Also note, when I am redeploying the agent gets further!
# (this suggests to me that it would be helpful to show an example of the contract address being taken from the skill.yaml file

# (however we still fail at the end)

# with this message

"""
info: [autonomous_hegician] requesting provide liquidity transaction...
INFO:aea.packages.tomrae.skills.option_monitoring:[autonomous_hegician] requesting provide liquidity transaction...
info: ******* Just about to get nonce for 0xD9EEA0deAC1cf2Ae88Cb2E9fBfa0De8A61F9CF7C type <class 'str'>
INFO:aea.packages.fetchai.contracts.ethpool:******* Just about to get nonce for 0xD9EEA0deAC1cf2Ae88Cb2E9fBfa0De8A61F9CF7C type <class 'str'>
info: ******* Just about to get instance 
INFO:aea.packages.fetchai.contracts.ethpool:******* Just about to get instance 
info: ************instance created:
INFO:aea.packages.fetchai.contracts.ethpool:************instance created:
error: [autonomous_hegician] An error occurred while processing the contract api request: ''name''.
ERROR:aea.packages.fetchai.connections.ledger:[autonomous_hegician] An error occurred while processing the contract api request: ''name''.
info: [autonomous_hegician] received ledger_api error message=Message(dialogue_reference=('6578b6b420e71ad8d649150da1bb3bc1686a712be12db544444661768d98d91b', 'bb48cab8b1e352867933c7e420dae065304fbe965bc039b0c2acbc8d2904d528') message_id=2 target=1 performative=error code=500 message='name' data=b'') in dialogue=Dialogue Label: 6578b6b420e71ad8d649150da1bb3bc1686a712be12db544444661768d98d91b_bb48cab8b1e352867933c7e420dae065304fbe965bc039b0c2acbc8d2904d528_fetchai/ledger:0.3.0_0xD9EEA0deAC1cf2Ae88Cb2E9fBfa0De8A61F9CF7C
info: get_raw_transaction( )
info: error( ).
"""

"\ninfo: [autonomous_hegician] requesting provide liquidity transaction...\nINFO:aea.packages.tomrae.skills.option_monitoring:[autonomous_hegician] requesting provide liquidity transaction...\ninfo: ******* Just about to get nonce for 0xD9EEA0deAC1cf2Ae88Cb2E9fBfa0De8A61F9CF7C type <class 'str'>\nINFO:aea.packages.fetchai.contracts.ethpool:******* Just about to get nonce for 0xD9EEA0deAC1cf2Ae88Cb2E9fBfa0De8A61F9CF7C type <class 'str'>\ninfo: ******* Just about to get instance \nINFO:aea.packages.fetchai.contracts.ethpool:******* Just about to get instance \ninfo: ************instance created:\nINFO:aea.packages.fetchai.contracts.ethpool:************instance created:\nerror: [autonomous_hegician] An error occurred while processing the contract api request: ''name''.\nERROR:aea.packages.fetchai.connections.ledger:[autonomous_hegician] An error occurred while processing the contract api request: ''name''.\ninfo: [autonomous_hegician] received ledger_api error message=Message(dialogue_ref

In [12]:

# To demonstrate using the underlyig library

In [4]:
from web3 import Web3

w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:7545'))
w3.isConnected()

True

In [177]:
ls

AutonomousHegician/  Pipfile       requirements.txt
packages/            Pipfile.lock  TestCases.ipynb


In [19]:
cd fetch_playground/fetch/

/home/tom/Desktop/Medium/fetch_playground/fetch


In [178]:

import sys
import time
import pprint
from web3 import Web3


In [179]:
with open("AutonomousHegician/contracts/ethpool/build/contracts/HegicETHPool.json", "r") as f:
    import json 
    c = json.loads(f.read())
    

address = "0x5Febf1fd482a3b525C2A3F54B21955FbD508A61B"
sum_add = Web3.toChecksumAddress(address)


deployed_contract = w3.eth.contract(address=sum_add,
                                    abi=c["abi"]
                                   )



In [180]:
tx_hash = deployed_contract.functions.provide(0).transact({'to': address,
                                                           'from': w3.eth.coinbase,
                                                           'value': 12345})
receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print("Transaction receipt mined:")
pprint.pprint(dict(receipt))
print("\nWas transaction successful?")
pprint.pprint(receipt["status"])

Transaction receipt mined:
{'blockHash': HexBytes('0x70bf2827fc47c87c6c1ecb6c0b9fee386af13cb3ea9425290163782cdb65c037'),
 'blockNumber': 300,
 'contractAddress': None,
 'cumulativeGasUsed': 46503,
 'from': '0xd45E8Cbb5A04C5e98CEb29d8ad9147Ee0D0F3Ec2',
 'gasUsed': 46503,
 'logs': [AttributeDict({'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0xa5c21865e0c560b45cb8d5482996a6def3dfcdaaec15996de721fa3f071bdbe7'), 'blockHash': HexBytes('0x70bf2827fc47c87c6c1ecb6c0b9fee386af13cb3ea9425290163782cdb65c037'), 'blockNumber': 300, 'address': '0x5Febf1fd482a3b525C2A3F54B21955FbD508A61B', 'data': '0x0000000000000000000000000000000000000000000000000000000000bc5ea8', 'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), HexBytes('0x000000000000000000000000d45e8cbb5a04c5e98ceb29d8ad9147ee0d0f3ec2')], 'type': 'mined'}),
          AttributeDict({'logIndex': 1, 'transac

In [195]:
with open("AutonomousHegician/contracts/ethpool/build/contracts/HegicCallOptions.json", "r") as f:
    import json 
    c = json.loads(f.read())
    

address = "0xfCc2b670A4704B10A5d0eF74106ff2edd2E8a453"
sum_add = Web3.toChecksumAddress(address)


deployed_contract = w3.eth.contract(address=sum_add,
                                    abi=c["abi"]
                                   )

tx_hash = deployed_contract.functions.fees(1111, 88888, 88888).transact({'to': address,
                                                                          'from': w3.eth.coinbase,
                                                                          'value': 12345})
receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print("Transaction receipt mined:")
pprint.pprint(dict(receipt))
print("\nWas transaction successful?")
pprint.pprint(receipt["status"])

ValueError: {'message': 'VM Exception while processing transaction: revert', 'code': -32000, 'data': {'stack': 'c: VM Exception while processing transaction: revert\n    at Function.c.fromResults (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:4:162210)\n    at e.exports (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:39:2190055)', 'name': 'c'}}

In [184]:
import pkg_resources 
pkg_resources.get_distribution("web3 ").version 

'5.11.1'

In [27]:
import math

In [101]:



from unittest import TestCase
    
    

class PremiumCalculator:
    def __init__(self,
                 period,
                 amount,
                 strike_price)
    
    self.period =period
    self.amount = amount
    self.strike_price = strike_price
    
    def __repr__(self):
        return f"For call option for {self.amount} over a period of {self.period} costs {self.fees()}"
    

class testPremiumCalculator(TestCase):
    
    
    
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def test_1(self):
        total = fees(period, amount, strike_price)
        print(total)
    
    
    
    def run(self):
        self.test_1()
        self.test_2()
        self.test_3()
        self.test_4()
        

tc = testPremiumCalculator(eth_val=0.1, period=86400, strike_price=9999)



tc.test_1()

# total 

SyntaxError: invalid syntax (<ipython-input-101-77a5bfa5a494>, line 48)

In [202]:
PRICE_DECIMALS = 100_000_000
implied_vol_rate = 5500


import decimal


def latest_answer():
    return 10000

option_type = "call"



def get_settlement_fee(amount):
    return amount / 100
    
    
def getPeriodFee(amount, period, strike, currentPrice):
    # we implement the call option
    return ((amount * math.sqrt(period) * implied_vol_rate * strike) / currentPrice) / PRICE_DECIMALS 
    raise
    
def getStrikeFee(amount, strike, current_price):
    if strike > current_price and option_type == "put":
        raise NotImplemented
    elif strike < current_price and option_type == "call":
        return ((current_price - strike) * amount) / current_price
    return 0














class PremiumCalculator:
    cost = None
    def __init__(self, period, amount, strike_price):
        self.period =period
        self.amount = amount
        self.strike_price = strike_price
        
    def __repr__(self):
        in_eth = Web3.fromWei(self.fees(self.period, self.amount, self.strike_price), 'ether')
        amt = Web3.fromWei(self.amount, "ether")
        strike_price = 391.75
        fees = self.fees(self.period, self.amount, self.strike_price)
        print(Web3.toWei(fees, "wei"))
        return f"For call option for {amt} over a period of {self.period} hours costs {Web3.fromWei(fees, 'ether') * 395}"
    
    def fees(self, period, amount, strike):
        current_price = latest_answer()
        settlementfee =get_settlement_fee(amount)
        periodFee = getPeriodFee(amount, period, strike, latest_answer())
        strikeFee = getStrikeFee(amount, strike, latest_answer())
        total =  strikeFee + periodFee
        return total



period = 24 * 3600 * 2
amount = Web3.toWei(0.1, unit="ether")
strike_price = 100
PC = PremiumCalculator(period, amount, strike_price)
print(PC)

strike_price = 100
PC = PremiumCalculator(period, amount, strike_price)
print(PC)

99022863070659900
For call option for 0.1 over a period of 172800 hours costs 39.114030912910662080
99022863070659900
For call option for 0.1 over a period of 172800 hours costs 39.114030912910662080


In [71]:
Web3.fromWei(total, "ether")

Decimal('0.063282986253926656')

In [174]:
Web3.toWei(amount, "wei")

1000000000000000000

In [153]:
def getStrikeFee(amount, strike, current_price):
    if strike > current_price and option_type == "put":
        raise NotImplemented
    elif strike < current_price and option_type == "call":
        return ((current_price - strike) * amount) / current_price
    return 0

option_type == "put"
getStrikeFee(1, 300, 305)

0.01639344262295082

In [191]:
int(strike_price) 

9000

In [203]:
tx_hash = deployed_contract.functions.fees(period, 1, 100000000000000).transact({'to': address,
                                                                          'from': w3.eth.coinbase,
                                                                          'value': 12345})

ValueError: {'message': 'VM Exception while processing transaction: revert', 'code': -32000, 'data': {'stack': 'c: VM Exception while processing transaction: revert\n    at Function.c.fromResults (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:4:162210)\n    at e.exports (/usr/local/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:39:2190055)', 'name': 'c'}}

In [194]:
period, amount, strike_price

(172800, 100000000000000000, 9000.0)